In [7]:
"""
    IntfromSting( X::String ) -> n
        Busca dentro del texto X si hay numeros que puedan ser convertido a entero.
        Se asume que ser'an un numero continuo de digitos para ser transformados a un entero
        Entonces regresa el entero formado desde el primer hasta el ultimo digito convertible.
"""
function IntfromSting( X::String )
    a = findfirst( isdigit.( collect( X ) ) )
    b = findlast( isdigit.( collect( X ) ) )
    if !isempty( a ) && !isempty( b )
        n = parse( Int, X[ a:b ] )
    else
        println(" There is no integers into this string ")
    end
    return n
end

using JLD
using MAT
using StatsBase
using Plots
using Statistics
using LinearAlgebra


PATHMain = raw"C:\Users\Siby\Desktop\Datos\Hipocampo\Phase_10";
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Hipocampo\BRWs\Phase_10.brw";

PATHInfo = joinpath( PATHMain , "Info" ); mkpath( PATHInfo );
PATHTimes = joinpath( PATHMain, "Times" ); mkpath( PATHTimes );
PATHFiguras = joinpath( PATHMain, "Figuras" ); 
FILEredundancia = joinpath( PATHInfo, "redundancia.jld" );
FILEAllInfoUnits = joinpath( PATHInfo, "AllInfoUnits_All.jld" );
FILEAllInfoUnits_no0 = joinpath( PATHInfo, "AllInfoUnits_no0.jld" );
FILEAllInfoUnits_noRed = joinpath( PATHInfo, "AllInfoUnits_noRed.jld" );
FILEBin = joinpath( PATHInfo, "Binarizacion.jld" );
FILEMeanSpikes = joinpath( PATHInfo, "MeanSpikes.mat" );
FIGUREMeanSpikes = joinpath( PATHFiguras, "MeanSpikes.svg" );

PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
push!( LOAD_PATH, PATHFunctions );

using F2023

FILEVars = joinpath( PATHInfo, "Variables.jld" );
FILESTIMES = FindContent( "times", FILEBRW );
Variables = load( FILEVars )[ "Variables" ];
nChs = Variables["nChs"];

factor = 42; # μm
d = 2; # ( Electrodos en la vecindad. ej. 1 == 8-vecindad, 2 == 24-vecindad )
pico = 19; # frame donde esta el valor minimo de voltaje, se puede sacar del "par" de cada espiga
factor_decaimiento = 2;
rsoma = 10; # μm, mitad del diametro promedio de una celula del tejido

j = 0; all_spikes = [ ]; all_cluster_class = [ ]; all_chs = [ ];

# -------------------------------------------------------------------------------------------------------- #
# Sacar los clusters y las espigas de los archivos times de matlab

j = 0; all_spikes = [ ]; all_cluster_class = [ ]; all_chs = [ ];

for k = 1:length( FILESTIMES )

    FILETimes = FILESTIMES[ k ]; temp = basename( FILETimes ); AUX = matread( FILETimes );

    n_cells = length( unique( AUX[ "cluster_class" ][ :, 1 ] ) );

    spikes = AUX[ "spikes" ];

    cluster_class = AUX[ "cluster_class" ];

    if n_cells > 1

        clus_num = unique( AUX[ "cluster_class" ][ :, 1 ] );

        for i = 1:n_cells

            j = j + 1;

            index_clus = findall( cluster_class[ :, 1 ] .== clus_num[ i ] );

            cc = cluster_class[ index_clus, : ];

            sp = spikes[ index_clus, : ];

            push!( all_spikes, sp ); push!( all_cluster_class, cc ); push!( all_chs, IntfromSting( temp ) );

        end
    else

        j = j + 1

        push!( all_spikes, spikes ); push!( all_cluster_class, cluster_class );

        push!( all_chs, IntfromSting( temp ) );

    end

end

save( FILEAllInfoUnits, "cluster_class", all_cluster_class, "spikes", all_spikes, "channels", all_chs );

println( "Sacar clusters excepto el 0, listo")

# quitar las espigas cluster 0 y sacar las cosas finales

all_times = [ ]; new_channels = [ ]; new_spikes = [ ];

for i = 1:length( all_cluster_class )

    temp = all_cluster_class[ i ];

    if Int( unique( temp[ :, 1 ] )[ ] ) != 0

        push!( all_times, temp[ :, 2 ] )

        push!( new_channels, all_chs[ i ] )

        push!( new_spikes, all_spikes[ i ] )

    end
end

save( FILEAllInfoUnits_no0, "times", all_times, "channels", new_channels, "spikes", new_spikes );

println( "Quitar cluster 0" )

# Eliminar la redundancia de la (d*2)-vecindad

EE = [ ];

for canal_central = 1:length( Canales )

    central = new_channels[ canal_central ];

    n_units = length( findall( new_channels .== central ) )

    for unit = 1:n_units # cuantas celulas hay en el canal central

        cc_central = all_times[ findall( new_channels .== central )[ unit ] ];

        sp_central = new_spikes[ findall( new_channels .== central )[ unit ] ];

        A, v = Neighbors( central, d ); # Vecinos del canal central

        vecinos = intersect( Canales, v ); # los que si pertenecen al clustering

        posiciones_vecinos = findall( new_channels .∈ [ vecinos ] );

        n_vecions = length( posiciones_vecinos );

        for n_vecion = 1:n_vecions

            vecino = new_channels[ posiciones_vecinos[ n_vecion ] ];

            cc_vecino = all_times[ posiciones_vecinos[ n_vecion ] ];

            sp_vecino = new_spikes[ posiciones_vecinos[ n_vecion ] ];

            maybes = intersect( cc_central, cc_vecino );

            posicion_sp_vecino = findall( cc_vecino .∈ [ maybes ] );

            posicion_sp_central = findall( cc_central .∈ [ maybes ] );

            voltage_central = sp_central[ posicion_sp_central, pico ];

            voltaje_vecino = sp_vecino[ posicion_sp_vecino, pico ];

            central_cartesian = [ ( d + 1 ), ( d + 1 ) ];

            vecino_cartesian = getindex.( findall( A .== vecino ), [ 1, 2 ] );

            dist =
                sqrt( ( central_cartesian[ 1 ] - vecino_cartesian[ 1 ] )^2
                    + ( central_cartesian[ 2 ] - vecino_cartesian[ 2 ] )^2 );

            distancia = dist*factor; # en micrometros

            for i = 1:length( maybes )

                x = [ voltage_central[ i ], voltaje_vecino[ i ] ];

                Φ1 = minimum( x );

                Φ2_1 = ( Φ1*rsoma )/( rsoma + distancia );

                Φ2_2 = Φ1 / ( ( distancia/20 )*factor_decaimiento );

                Φ2_real = maximum( x ); mas_menos = abs( Φ2_1 ) - abs( Φ2_2 );

                posible_valor = [
                    ( minimum( [ Φ2_1, Φ2_2 ] ) + mas_menos ), ( minimum( [ Φ2_1, Φ2_2 ] ) - mas_menos ) ];

                if maximum( posible_valor )> Φ2_real > minimum( posible_valor )

                    eliminar = findall( x .== Φ2_real )[ 1 ];
                    # 1 = voltaje central
                    # 2 = voltaje vecino

                    e1 = vcat(
                        posicion_sp_central[ i ], central, findall( new_channels .== central )[ unit ],
                        x[ 1 ] );

                    e2 = vcat( posicion_sp_vecino[ i ], vecino, posiciones_vecinos[ n_vecion ], x[ 2 ] );

                    if eliminar == 1

                        push!( EE, vcat( distancia, e1, e2 ) )

                    elseif eliminar == 2

                        push!( EE, vcat( distancia, e2, e1 ) )

                    end
                    # else si son distintas, no hagas nada
                end

            end

        end

    end

end

EE = Array( hcat( EE... )' ); save( FILEredundancia, "redundancia",  EE );

# se va, se queda

# espiga_no, canal_no, posicion_no, voltaje_no
# espiga_si, canal_si, posicion_si, voltaje_si

finales_eliminar = Int.( unique( EE[ :, 2:4 ], dims = 1 ) );

a = finales_eliminar[ :, 3 ]; definitivos = findall.( isequal.( unique( a ) ), [ a ] );

for j = 1:length( definitivos )

    p = finales_eliminar[ definitivos[ j ][ 1 ], 3 ];

    q = finales_eliminar[ definitivos[ j ], 1 ];

    new_spikes[ p ] = new_spikes[ p ][ 1:end .∉ [ q ], : ];

    all_times[ p ] = all_times[ p ][ 1:end .∉ [ q ], : ];

end

save( FILEAllInfoUnits_noRed, "cluster_class", all_times, "spikes", new_spikes, "channels", new_channels );

println( "Eliminar la n-vecindad: listo " );

# sacar espiga promedio para clustering

all_MS = zeros( length( new_spikes ), size( new_spikes[ 1 ], 2 ) ); thr = 0.5;

@time for i = 1:length( new_spikes )

    spikes = new_spikes[ i ];
    
    all_MS[ i, : ] = vec( mean( spikes, dims = 1 ) );
    #all_MS[ i, : ] = vec( meanspike( spikes, thr ) );
    
end

matwrite( FILEMeanSpikes, Dict( "data" => all_MS ) );

# -------------------------------------------------------------------------------------------------------- #

@inline distancia( x1, x2, y1, y2, factor ) = ( sqrt( ( x2 - x1 )^2 + ( y2 - y1 )^2 ) )*factor;
@inline t( v, d ) = ( ( d / 1000000 ) / v ); # v(m/s), d(μm)

FILEAllInfoUnits_noRed = joinpath( PATHInfo, "AllInfoUnits_noRed.jld" );

TEMP = load( FILEAllInfoUnits_noRed );

NRecFrames = Int(Variables[ "NRecFrames" ]/45);
SamplingRate = Variables[ "SamplingRate" ];
segundos = ceil( Int, NRecFrames/SamplingRate );

factor = 42; # μm
dims_chip = 2.7; # mm
#restriccion = floor( sqrt( dims_chip^2 + dims_chip^2 )*1000, digits = 3 ); # μm
restriccion = 1000; # μm
times = TEMP[ "cluster_class" ]; spikes = TEMP[ "spikes" ]; channels = Int.( TEMP[ "channels" ] );

    Layout = reverse( reshape( collect( 1:4096 ), 64, 64 )', dims = 1 );

grupos, loose = Get_Groups( unique(channels) )

Frec = size.( spikes, 1) / segundos;

Ti = F2023.FindAllThrs( Frec )

aux = load(raw"C:\Users\Siby\Desktop\Datos\Hipocampo\Phase_10\Info\Analitics\Ws\BIN16.jld");

aux = aux["Data"];

aux = Float64.(hcat(aux...));

sigma = 3;

P = Array{ Plots.Plot{ Plots.GRBackend } }( undef, 30 );
Q = Array{ Plots.Plot{ Plots.GRBackend } }( undef, 30 );
for a = 1:30
    data = aux[ :, a ]; 
    P[ a ] = Zplot( RemoveInfs( log.( abs.( data ) ) ), "W", :bluesreds ); 
    hg = convgauss( sigma, data );
    try
        data = RemoveInfs( log.( abs.( vec( hg ) ) ) );
        Q[ a ] = Zplot( data, "W", :bluesreds ); 
    catch e
        Q[a] = P[a]
    end
end

X = sum(abs.(aux[:,[4,6,16]]), dims = 2);
Zplot(RemoveInfs(log.(aux[:,4])), "W",:bluesreds)

W = log.(aux[:,16]);

T1 = FindAllThrs(W);

Zplot( findall(W .> T1[9]), "0")



plot( P..., layout = ( 10, 3 ), wsize = ( 800, 1200 ), cbar = :none, titlefontsize = 10 )

savefig("P.svg")

plot( Q..., layout = ( 10, 3 ), wsize = ( 800, 1200 ), cbar = :none, titlefontsize = 10 )

savefig("Q.svg")

pwd()



















using Plots
function FigureGroups( grupos::Vector, loose::Vector = [ ], cm = :twilight )
    Z = zeros( Int, 4096 );
    for i = 1:size( grupos, 1 )
        Z[ grupos[ i ] ] .= floor( Int, log( length( grupos[ i ] ) ) ) + 2 ;
    end
    Z[ Int.( loose ) ] .= 1
    Z = reverse( reshape( Z, 64, 64 )', dims = 1 )
    F = heatmap(
        Z,
        aspect_ratio = 1,
        c = cm,
        axis = ( [ ], false ),
        wsize = ( 400, 400 ),
        cbar = :none );
    return F
end

"""
    Get_Groups( W::Vector{Int64} ) -> grupos::Vector{Any}, loose::( Vector{Any} )
       Groups of adjacent channels are formed from the initial indexes W. Those channels separated from the bulk of the others are considered loose ones.
"""
function Get_Groups( W::Vector{Int64} )
    grupos = [ ];
    for i in W
        _, vecinos = Neighbors( i, 1 );
        grupo = sort( intersect( vecinos, W ) );
        if isempty( grupo )
            push!( grupos, i )
        else
            push!( grupos, vcat( grupo, i ) )
        end
    end
    loose = grupos[ findall( length.( grupos ) .== 1 ) ];
    deleteat!( grupos, findall( length.( grupos ) .== 1 ) );
    a = length( grupos ); grupos = reverberation( grupos ); b = length( grupos );
    while a != b
        a = length( grupos ); grupos = reverberation( grupos ); b = length( grupos );
    end
    return grupos, loose
end

# -------------------------------------------------------------------------------------------------------- #
"""
    reverberation( grupos::Vector{Any} ) -> more_groups::Vector{Any}
        Intermediate step for Get_Groups function
"""
function reverberation( grupos::Vector{Any} )
    adjoining_channels = 0
    for i in grupos
        adjoining_channels = vcat( adjoining_channels, i )
    end
    adjoining_channels = adjoining_channels[ adjoining_channels .!= 0 ];
    adjoining_channels = unique( adjoining_channels );
    more_groups = [ ]
    for i in adjoining_channels
        temporal = 0
        for j = 1:length( grupos )
            if !isempty( findall( grupos[ j ] .== i ) )
                temporal = vcat( temporal, grupos[ j ] );
            end
        end
        temporal = temporal[ temporal .!= 0 ];
        new_group = sort( unique( temporal ) );

        if !isempty( new_group )
            if !isempty( more_groups )
                temp = last( more_groups );
                if !isequal( temp, new_group )
                    push!( more_groups, new_group );
                end
            else
                push!( more_groups, new_group );
            end
        end
    end
    return more_groups
end



IntfromSting